In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

%matplotlib inline

In [15]:
unreliable = pd.DataFrame()
for fname in ['bias', 'conspiracy', 'fake', 'hate', 'pol_bogus']:
    unreliable = unreliable.append(pd.read_csv('cleaned/{}_cleaned.csv'.format(fname)))   

In [4]:
unreliable.groupby('domain').count().iloc[:,0].sort_values()

domain
usanewsflash.com                     1
uspoln.com                           3
usafirstinformation.com              3
countdowntozerotime.com              3
goneleft.com                         4
viralliberty.com                     5
glaringhypocrisy.com                 5
newslogue.com                        5
dataasylum.com                       6
winningdemocrats.com                 9
thelastgreatstand.com                9
flashnewscorner.com                 10
sonsoflibertyradio.com              11
sheepkillers.com                    12
redcountry.us                       12
enhlive.com                         12
donaldtrumpnews.co                  13
onepoliticalplaza.com               13
readconservatives.news              15
jamesrgrangerjr.com                 15
yesimright.com                      18
nasamoonhoax.com                    20
usadosenews.com                     21
americanpatriotdaily.com            27
newcoldwar.org                      35
platosguns.com    

In [17]:
unreliable = unreliable.loc[:, ['id', 'domain', 'content']]

In [20]:
unreliable = unreliable.set_index('id')

In [ ]:
# take 100,000 from dailykos to balance right-leaning orgs